# TUTORIAL: ¡SCIKIT-LEARN DESDE CERO!

En este tutorial veremos paso a paso todos los elementos básicos que usualmente hay que tener en cuenta para crear, entrenar, validar y poner a prueba prácticamente cualquier modelo de Machine Learning clásico usando esta librería.

Contenido:

1. [¿Qué es Scikit-Learn?](#scrollTo=Mwb3z4nhqir3&line=1&uniqifier=1)
2. [El flujo de trabajo convencional en Scikit-Learn](#scrollTo=l7BA3TMarh4z&line=1&uniqifier=1)
3. [Pre-procesamiento: generar particiones](#scrollTo=Eqs5h_Ijv6Mc&line=1&uniqifier=1)
4. [Pre-procesamiento: transformadores](#scrollTo=Xn2shF8Vz-UQ&line=1&uniqifier=1)
5. [Crear, entrenar y validar el modelo: estimadores](#scrollTo=BtFsAQYJg-JL&line=1&uniqifier=1)
6. [*Pipelines*](#scrollTo=Zy6KK9A7jtdr&line=1&uniqifier=1)


## 1. ¿Qué es Scikit-Learn?

> Scikit-learn es una librería de Machine Learning de código abierto que contiene herramientas para **pre-procesamiento** de datos, **entrenamiento** y generación de **predicciones** con diferentes modelos **clásicos** y **selección y validación** de modelos, entre otras.

Este es el panorama general de los diferentes algoritmos de Machine Learning implementados en Scikit-Learn:

<figure>
<img src="https://scikit-learn.org/1.4/_static/ml_map.png" style="width:100%">
<figcaption align = "center"> Los diferentes algoritmos de Scikit-Learn (tomada del sitio web oficial de la librería) </figcaption>
</figure>

## 2. El flujo de trabajo convencional en Scikit-Learn

![](https://drive.google.com/uc?export=view&id=1uyNbPoI1zX8BG4ISyD9Kud54ydI67mqu)

En esencia:

- Podemos implementar modelos para tareas de aprendizaje supervisado (como clasificación o regresión) o para aprendizaje NO supervisado (como *clustering*)
- En el **pre-procesamiento** generalmente debemos **generar particiones** de los sets de datos (en entrenamiento, validación y prueba) o realizar **transformaciones** de los datos (como el escalamiento)
- Luego **creamos una instancia del algoritmo**, **entrenamos** el modelo y lo **validamos** con los sets de entrenamiento, validación y/o prueba
- Finalmente, el modelo está listo para **generar predicciones**

Veamos de forma práctica los elementos básicos de cada una de estas etapas.

## 3. Pre-procesamiento: generar particiones

Una tarea común consiste en dividir el set de datos en los sets de entrenamiento, validación y prueba.

Esto lo podemos hacer con la función `train_test_split`.

Para entender cómo usarla comencemos leyendo el set de datos `particiones_datos_balanceados.npz` (arreglo de NumPy):



En este caso tenemos un set de datos supervisado, con:

- `X`: el arreglo de entrada al modelo (20 datos x 3 características)
- `Y`: la variable que deberá aprender a predecir el modelo (20 datos)

> **Nota importante:** los arreglos siempre deben estar dimensionados como *n_datos x n_características* (X) y *n_datos* (Y)

Supongamos que queremos realizar la partición con estas proporciones:

- Entrenamiento: 60%
- Validación: 20%
- Prueba: 20%

En este caso debemos usar `train_test_split` dos veces:

- En el primer paso partimos el set de datos en 2: 60% (entrenamiento) y 40% (resto)
- En el segundo paso partimos el set de datos restante en 2 mitades: 50% (20% del dataset original, validación) y 50% (20% del dataset original, prueba)

Veamos cómo implementar esta partición:

En el caso anterior lo que hace `train_test_split` es:

1. Mezclar aleatoriamente el set de datos
2. Generar las particiones con las proporciones correspondientes

Es decir, en últimas crea cada subset usando **muestreo aleatorio**.

Este muestreo aleatorio es adecuado si por ejemplo estamos implementando un clasificador y las categorías están balanceadas, es decir, tienen más o menos la misma proporción de una categoría o de otra (como es el caso del ejemplo anterior).

Sin embargo, este muestreo aleatorio no es adecuado si tenemos datos desbalanceados, es decir con una proporción mayor de una categoría que de otra.

Por ejemplo, leamos el dataset `particiones-datos-desbalanceados.npz` y veamos la proporción de los datos:

Claramente es un set desbalanceado: 85% categoría "0" y 15% categoría "1".

Así que si hacemos la partición y queremos por ejemplo implementar un modelo de detección de anomalías **debemos preservar estas proporciones**.

Esto se logra usando `train_test_split` pero usando un **muestreo estratificado**. En este caso simplemente usamos el argumento `stratify = Y` para que al hacer el muestreo la función tenga en cuenta las proporciones presentes en el arreglo `Y`:

Vemos que el muestreo estratificado intenta mantener las proporciones de cada categoría al generar cada uno de los subsets.

Y con esto ya hemos visto cómo implementar una primera fase de pre-procesamiento.

Veamos una segunda fase que es el uso de **transformadores**.

## 4. Pre-procesamiento: transformadores

> Permiten transformar los datos: escalar (`RobustScaler`, `MinMaxScaler`, `StandardScaler`), codificar (`LabelEncoder`, `OneHotEncoder`) o reducir (`PCA`), entre otras

Los pasos para usar un transformador son:

1. Crear una instancia del transformador
2. Usar el método `fit_transform()` para transformar el set de entrenamiento
3. Usar el método `transform()` para transformar los sets de validación y prueba


Por ejemplo, veamos los rangos de valores de cada columna en los sets de entrenamiento, validación y prueba (`x_train`, `x_val` y `x_test`):

Vemos que las variables (columnas) tienen diferentes rangos: -3 a 3, -4 a 5 y -9 a 9 aproximadamente.

Así que un tipo de pre-procesamiento sería, por ejemplo, escalar cada columna al mismo rango de valores antes de llevar los datos al modelo.

Por ejemplo, supongamos que haremos el escalamiento en el rango de -1 a 1 para lo cual podemos usar el transformador `MinMaxScaler`.

Veamos cada uno de los pasos a llevar a cabo. En primer lugar creamos una instancia del transformador:

El segundo paso es usar el método `fit_transform()` aplicado sobre el set de entrenamiento (`x_train`). Este método:

- Calculará y almacenará en la instancia los mínimos y máximos de cada columna de `x_train`
- Y luego escalará `x_train` al rango de -1 a 1 usando los máximos y mínimos recién calculados

Veamos este segundo paso:

Y verifiquemos que `x_train_s` contiene ahora los datos escalados al rango de -1 a 1:

El tercer paso es tomar el escalador (`scaler`) y usar el método `transform()` para transformar (escalar) los sets de validación (`x_val`) y prueba (`x_test`):

Verifiquemos que ahora el rango de valores en estos dos sets está entre -1 y 1:

Vemos que se realiza el escalamiento pero los valores mínimos y máximos no son exactamente -1 y 1. Esto debido a que el escalamiento se realiza **con base en los valores máximos y mínimos del set de entrenamiento** que no necesariamente son iguales a los de los sets de validación y prueba.

## 5. Crear, entrenar y validar el modelo: estimadores

En Scikit-Learn los modelos se denominan estimadores.

La secuencia de uso es la siguiente:

1. Crear una instancia del estimador
2. Entrenar el modelo con el set de entrenamiento y el método `fit()`
3. Validar el modelo con los sets de entrenamiento y validación usando el método `score()`
4. Poner a prueba el modelo con el set de prueba y usando los métodos `predict()` y `score()`

Veamos en detalle cada uno de estos pasos. Supongamos que tomaremos el set de datos que hemos venido usando para crear, entrenar, validar y poner a prueba un Bosque Aleatorio.

El primer paso es crear una instancia de este estimador (`RandomForestClassifier`):

El segundo paso es entrenarlo. Para ello usamos el método `fit()` y le presentamos como argumentos el set de entrenamiento (`x_train`, `y_train`):

El tercer paso es validar el modelo. Esto quiere decir que la idea es comparar el desempeño con los sets de entrenamiento y validación, para determinar si hay o no *overfitting* u *underfitting*.

El desempeño es simplemente una métrica que cuantifica qué tan bien lo esta haciendo el modelo.

Verifiquemos en este caso cuál es el desempeño usado por defecto por el bosque aleatorio:

Vemos que el desempeño se está midiendo con la exactitud promedio (*mean accuracy*).

Así que calculemos el desempeño con los sets de entrenamiento y validación:

En este caso vemos que el modelo tiene *overfitting*, pues alcanza un 100% de exactitud con el set de entrenamiento y tan sólo un 50% con el set de prueba.

En realidad es de esperar pues tenemos poquísimos datos y no hemos modificado ningún parámetro por defecto del modelo.

En una situación real deberíamos recolectar más datos y re-entrenar el modelo, posiblemente afinando sus hiperparámetros (pero esto será tema de un tutorial más avanzado).

El cuarto y último paso es poner a prueba el modelo. Para ello podemos primero ver el *score* con el set de prueba:

Que sigue siendo del 50% por los mismos motivos mencionados anteriormente.

Y, suponiendo que estamos conformes con estos resultados, lo que faltaría sería generar predicciones usando el método `predict()`.

Tomemos nuevamente el set de prueba y generemos predicciones con el modelo entrenado:

Y como tenemos tan pocos datos podemos imprimir el comparativo entre las categorías reales (almacenadas en `y_test`) y las categorías predichas (almacenadas en `y_pred`):

Y vemos que en efecto de los 4 datos sólo dos (los dos últimos) son clasificados correctamente.

## 6. *Pipelines* (tuberías????)

Es posible combinar transformadores y estimadores en un sólo objeto: una *pipeline*.

Una *pipeline* nos permite hacer lo mismo que con los bloques separados, pero tiene ciertas ventajas:

1. El código es más compacto
2. Evita lo que se conoce como la fuga de datos (*data leakage*): que los datos de validación sean "vistos" por el modelo cuando hacemos el entrenamiento

Para hacer un comparativo veamos primero cómo sería el flujo completo de trabajo sin *pipelines*: